# import

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")

## Define tools

In [2]:
# from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from elasticsearch import Elasticsearch
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel, Field  # <-- Uses v1 namespace
from langchain.tools import StructuredTool

In [3]:

class SearchToolInput(BaseModel):
    """Input for the index show data tool."""

    index_name: str = Field(
        ..., description="The name of the index for which the data is to be retrieved. in this case indexname will be " + ES_INDEX_NAME
    )
    query: str = Field(..., description="The ElasticSearch JSON query used to filter all hits. Should use the _source field if possible to specify required fields.")
    from_: int = Field(
        ..., description="The record index from which the query will start"
    )
    size: int = Field(
        ...,
        description="How many records will be retrieved from the ElasticSearch query",
    )

### Embedding 정의

In [4]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


### ES retirever 정의

In [5]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
vector_store = ElasticsearchStore(
    embedding=embeddings,
    index_name=ES_INDEX_NAME,
    es_connection=es
)
retriever = vector_store.as_retriever(search_kwargs={"k": 10})
# retriever.as_tool()
search_results = retriever.invoke("애플")
print(search_results[0])
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_69480/3560551961.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


page_content='사람들도 다 바꿀 가능성이 더 높아진다?◎ 이형수 > 네네. 맞습니다.◎ 진행자 > 그래서 다시 한번 매출이 튀어 올라갈 거다라고 보는군요.◎ 이형수 > 그런 기대감이 이제 어젯밤부터 반영이 되기 시작한 거 같아요.◎ 진행자 > 그렇군요. 발표된 내용들이 이것저것 또 추가로 있을 텐데 보시기에는 또 어떤 특징들을 이번에 담고 있습니까? 애플의 신제품 내지는 앞으로 애플의 계획 전략을 들여다보면.◎ 이형수 > 결국 이제 애플도 결국 AI라는 게 범용 AI 우리가 이제 이야기하는 초지능 아티피셜 제너럴 인텔리전스라는 거를 결국 이제 세상을 움직이는 OS라고 생각하거든요. 기존에 우리가 이제 오퍼레이팅 시스템이라고 하면 특정 디바이스 특정 기기에 제한된 건데. AI라는 거는 사실 세상을 움직이는 OS가 될 수 있거든요. 여기서 이제 애플도 이거를 시작으로 우리도 이 한 자리를 차지하겠다라는 생각인 것 같고요. 그게 가능한 거는 결국 이 스마트폰 PC 이런 것들을 통해서 단순히 이제 OS뿐만 아니라 하드웨어 반도체 설계, 하드웨어 소프트웨어 생태계같이 수직 계열화하고 있는 회사가 애플 제외하면 거의 없거든요.◎ 진행자 > 어디는 칩만 만들고 어디는 인공지능 못 만들고 어디는 또 그거 갖다 서비스하고 그러고 있는데 애플은 칩부터 다 만든다.◎ 이형수 > 네네 맞습니다.◎ 진행자 > 인공지능만 살짝 지금 빌려온 거다?◎ 이형수 > 그렇죠. 지금은 뒤처져 있지만 빠른 속도로 우리가 추월할 수 있다라고 생각하는 것 같고요. 그리고 이제 실제로 이런 것들을 위해서 많은 준비들을 하고 있는데 그동안 애플이 아이 클라우드 같은' metadata={'id': 639052, 'hash_key': '29d1f0a79f8f9c9c326378ce9e6b83d1', 'title': '[손에잡히는경제] 이형수 "애플, AI 자체개발 포기? 점차 내재화할 것..자체 클라우드 사업도"', 'created_date': '2024-06-12T12:50:15', 'portal

In [6]:
def create_es_search_tool():
      return StructuredTool(name="elastic_index_search_tool",
                            func=retriever.get_relevant_documents, 
                            args_schema=SearchToolInput)
tools = [create_es_search_tool()]
# tools = [retriever.as_tool()]

In [7]:
# from langchain.tools.retriever import create_retriever_tool

# retrieve_tool = create_retriever_tool(
#     retriever,
#     name="elastic_index_search_tool",
#     description="return search result from user input query"
# )
# tools = [retrieve_tool]

## define llm

In [8]:
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [9]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

### agentic

In [10]:
model_with_tools = llm.bind_tools(tools)

In [11]:
response = model_with_tools.invoke([HumanMessage(content=f"{ES_INDEX_NAME} 색인에서 삼성전자 뉴스를 찾아줘!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'elastic_index_search_tool', 'args': {'index_name': 'news_article_embedding', 'query': {'query': {'match': {'content': '삼성전자'}}, '_source': ['title', 'content', 'date']}, 'from_': 0, 'size': 5}, 'id': 'call_YgpPTVnpkUnO9uyRBYKIALZX', 'type': 'tool_call'}]


# Create Agent

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=f"2024년 애플에 대한 긍정적인 뉴스를 찾아서, 개별 기사에서 긍정적인 사유를 설명하고,  개별 모든 뉴스에 대해서 지수를 점수로 매겨서 리스트로 표시해")]}
)
response["messages"]

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[HumanMessage(content='2024년 애플에 대한 긍정적인 뉴스를 찾아서, 개별 기사에서 긍정적인 사유를 설명하고,  개별 모든 뉴스에 대해서 지수를 점수로 매겨서 리스트로 표시해', id='0b1b1dc2-054b-40f5-a4ba-264cef1be069'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7ETLHOlHWQjTEcBt4d3XYX3y', 'function': {'arguments': '{"index_name": "news_article_embedding", "query": "{\\"query\\":{\\"bool\\":{\\"must\\":[{\\"term\\":{\\"company\\":\\"Apple\\"}},{\\"term\\":{\\"year\\":2024}},{\\"match\\":{\\"sentiment\\":\\"positive\\"}}]}}}", "from_": 0, "size": 10}', 'name': 'elastic_index_search_tool'}, 'type': 'function'}, {'id': 'call_g2HS3RwbMylMTLzsyw1LWC6v', 'function': {'arguments': '{"index_name": "news_article_embedding", "query": "{\\"query\\":{\\"bool\\":{\\"must\\":[{\\"term\\":{\\"company\\":\\"Apple\\"}},{\\"term\\":{\\"year\\":2024}},{\\"match\\":{\\"sentiment\\":\\"positive\\"}}]}}}", "from_": 10, "size": 10}', 'name': 'elastic_index_search_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens

In [15]:
response["messages"][-1]

AIMessage(content='다음은 2024년에 애플에 대한 긍정적인 뉴스를 모은 리스트와 각 뉴스의 긍정적인 사유 및 지수 점수입니다.\n\n1. **[성공예감] 실리콘밸리를 지배할 2024년의 키워드는 - 송이라 기자(서울경제신문)**\n   - **긍정적인 사유:** 애플이 IT 기술과 투자에서 주도적인 역할을 하고 있으며, 특히 글로벌 금융위기 이후 경제 회복의 중심에 서 있다는 평가.\n   - **지수 점수:** 9.0\n\n2. **애플카 나온다더니 감감무소식…애플, 신사업 전략은?**\n   - **긍정적인 사유:** 애플이 새로운 제품을 통해 지속적인 성장 가능성을 보여주고 있으며, 투자자들로부터 높은 기대를 받고 있음.\n   - **지수 점수:** 8.5\n\n3. **애플·TSMC 함께 날았다…뉴욕증시 반등 성공**\n   - **긍정적인 사유:** 애플의 주가가 급등하며 뉴욕 증시의 반등을 이끌어냈고, 특히 중국 시장에서의 매출 증가가 긍정적인 영향을 미침.\n   - **지수 점수:** 9.2\n\n4. **애플 "큰 실수" 주춤한 사이…삼성 \'갤S24\' AI폰 선공 나선다**\n   - **긍정적인 사유:** 애플의 주가가 큰 폭으로 상승하였으며, AI 기술을 중심으로 하는 신제품 출시 계획이 주목받고 있음.\n   - **지수 점수:** 8.0\n\n5. **[트렌딩 핫스톡] BofA, 애플 투자 의견 매수·목표주가 225달러로 상향**\n   - **긍정적인 사유:** Bank of America가 애플의 주가 목표를 상향 조정하며 긍정적인 투자 의견을 제시함.\n   - **지수 점수:** 8.8\n\n6. **[문남중의 금요시장] 미 증시 주도주, 전기차에서 AI로…서학개미들의 선택은?**\n   - **긍정적인 사유:** 애플이 AI 중심의 기술 혁신을 통해 시장의 주요 주도주로 부상하고 있음.\n   - **지수 점수:** 8.7\n\n7. **[2024 증시전망] 목대전 KCGI운용 CIO “자사주만 소각해도 코